<a href="https://colab.research.google.com/github/victor-mthemba/regression_repo/blob/master/The_predict_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Making a model that predicts the delivery time of packages from pick up to the final destination considering various factors.

## Importing the libraries

In [0]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

## Importing the dataset

In [0]:
train_df = pd.read_csv("Train.csv")
test_df = pd.read_csv("Test.csv")
riders_df = pd.read_csv("Riders.csv")
sample_df = pd.read_csv("SampleSubmission.csv")

In [33]:
train_df.head()

,Order No,User Id,Vehicle Type,Platform Type,Personal or Business,Placement - Day of Month,Placement - Weekday (Mo = 1),Placement - Time,Confirmation - Day of Month,Confirmation - Weekday (Mo = 1),Confirmation - Time,Arrival at Pickup - Day of Month,Arrival at Pickup - Weekday (Mo = 1),Arrival at Pickup - Time,Pickup - Day of Month,Pickup - Weekday (Mo = 1),Pickup - Time,Arrival at Destination - Day of Month,Arrival at Destination - Weekday (Mo = 1),Arrival at Destination - Time,Distance (KM),Temperature,Precipitation in millimeters,Pickup Lat,Pickup Long,Destination Lat,Destination Long,Rider Id,Time from Pickup to Arrival
0,Order_No_4211,User_Id_633,Bike,3,Business,9,5,9:35:46 AM,9,5,9:40:10 AM,9,5,10:04:47 AM,9,5,10:27:30 AM,9,5,10:39:55 AM,4,20.4,NaN,-1.317755,36.830370,-1.300406,36.829741,Rider_Id_432,745
1,Order_No_25375,User_Id_2285,Bike,3,Personal,12,5,11:16:16 AM,12,5,11:23:21 AM,12,5,11:40:22 AM,12,5,11:44:09 AM,12,5,12:17:22 PM,16,26.4,NaN,-1.351453,36.899315,-1.295004,36.814358,Rider_Id_856,1993
2,Order_No_1899,User_Id_265,Bike,3,Business,30,2,12:39:25 PM,30,2,12:42:44 PM,30,2,12:49:34 PM,30,2,12:53:03 PM,30,2,1:00:38 PM,3,NaN,NaN,-1.308284,36.843419,-1.300921,36.828195,Rider_Id_155,455
3,Order_No_9336,User_Id_1402,Bike,3,Business,15,5,9:25:34 AM,15,5,9:26:05 AM,15,5,9:37:56 AM,15,5,9:43:06 AM,15,5,10:05:27 AM,9,19.2,NaN,-1.281301,36.832396,-1.257147,36.795063,Rider_Id_855,1341
4,Order_No_27883,User_Id_1737,Bike,1,Personal,13,1,9:55:18 AM,13,1,9:56:18 AM,13,1,10:03:53 AM,13,1,10:05:23 AM,13,1,10:25:37 AM,9,15.4,NaN,-1.266597,36.792118,-1.295041,36.809817,Rider_Id_770,1214


In [34]:
test_df.head()

,Order No,User Id,Vehicle Type,Platform Type,Personal or Business,Placement - Day of Month,Placement - Weekday (Mo = 1),Placement - Time,Confirmation - Day of Month,Confirmation - Weekday (Mo = 1),Confirmation - Time,Arrival at Pickup - Day of Month,Arrival at Pickup - Weekday (Mo = 1),Arrival at Pickup - Time,Pickup - Day of Month,Pickup - Weekday (Mo = 1),Pickup - Time,Distance (KM),Temperature,Precipitation in millimeters,Pickup Lat,Pickup Long,Destination Lat,Destination Long,Rider Id
0,Order_No_19248,User_Id_3355,Bike,3,Business,27,3,4:44:10 PM,27,3,4:44:29 PM,27,3,4:53:04 PM,27,3,5:06:47 PM,8,NaN,NaN,-1.333275,36.870815,-1.305249,36.822390,Rider_Id_192
1,Order_No_12736,User_Id_3647,Bike,3,Business,17,5,12:57:35 PM,17,5,12:59:17 PM,17,5,1:20:27 PM,17,5,1:25:37 PM,5,NaN,NaN,-1.272639,36.794723,-1.277007,36.823907,Rider_Id_868
2,Order_No_768,User_Id_2154,Bike,3,Business,27,4,11:08:14 AM,27,4,11:25:05 AM,27,4,11:33:20 AM,27,4,11:57:54 AM,5,22.8,NaN,-1.290894,36.822971,-1.276574,36.851365,Rider_Id_26
3,Order_No_15332,User_Id_2910,Bike,3,Business,17,1,1:51:35 PM,17,1,1:53:27 PM,17,1,2:02:41 PM,17,1,2:16:52 PM,5,24.5,NaN,-1.290503,36.809646,-1.303382,36.790658,Rider_Id_685
4,Order_No_21373,User_Id_1205,Bike,3,Business,11,2,11:30:28 AM,11,2,11:34:45 AM,11,2,11:47:19 AM,11,2,11:56:04 AM,6,24.4,NaN,-1.281081,36.814423,-1.266467,36.792161,Rider_Id_858


In [35]:
riders_df.head()

,Rider Id,No_Of_Orders,Age,Average_Rating,No_of_Ratings
0,Rider_Id_396,2946,2298,14.0,1159
1,Rider_Id_479,360,951,13.5,176
2,Rider_Id_648,1746,821,14.3,466
3,Rider_Id_753,314,980,12.5,75
4,Rider_Id_335,536,1113,13.7,156


In [36]:
sample_df.head()

,Order_No,Time from Pickup to Arrival
0,Order_No_19248,567.0
1,Order_No_12736,4903.0
2,Order_No_768,5649.0
3,Order_No_15332,NaN
4,Order_No_21373,NaN


## Merging Tables

In [0]:
df_train = pd.merge(train_df, riders_df,on = 'Rider Id', how= 'inner')

In [0]:
df_test =  pd.merge(train_df, riders_df,on = 'Rider Id', how= 'inner')

## Remove unnecessary columns

In [0]:
remove_list_train = ['Order No', 'User Id', 'Platform Type', 'Arrival at Destination - Day of Month','Placement - Weekday (Mo = 1)','Confirmation - Weekday (Mo = 1)','Arrival at Pickup - Weekday (Mo = 1)',
                          'Pickup - Weekday (Mo = 1)','Arrival at Destination - Weekday (Mo = 1)','Vehicle Type', 'Temperature', 'Precipitation in millimeters' 
                                ]
remove_list_test = ['Order No', 'User Id', 'Platform Type', 'Arrival at Destination - Day of Month','Placement - Weekday (Mo = 1)','Confirmation - Weekday (Mo = 1)','Arrival at Pickup - Weekday (Mo = 1)',
                          'Pickup - Weekday (Mo = 1)','Arrival at Destination - Weekday (Mo = 1)','Vehicle Type', 'Temperature', 'Precipitation in millimeters' 
                                ]

In [0]:
df_train = df_train.drop(remove_list_train, axis = 1)

In [41]:
df_train.head()

,Personal or Business,Placement - Day of Month,Placement - Time,Confirmation - Day of Month,Confirmation - Time,Arrival at Pickup - Day of Month,Arrival at Pickup - Time,Pickup - Day of Month,Pickup - Time,Arrival at Destination - Time,Distance (KM),Pickup Lat,Pickup Long,Destination Lat,Destination Long,Rider Id,Time from Pickup to Arrival,No_Of_Orders,Age,Average_Rating,No_of_Ratings
0,Business,9,9:35:46 AM,9,9:40:10 AM,9,10:04:47 AM,9,10:27:30 AM,10:39:55 AM,4,-1.317755,36.830370,-1.300406,36.829741,Rider_Id_432,745,1637,1309,13.8,549
1,Personal,18,3:41:17 PM,18,3:41:30 PM,18,4:07:16 PM,18,4:13:37 PM,5:01:43 PM,20,-1.326774,36.787807,-1.356237,36.904295,Rider_Id_432,2886,1637,1309,13.8,549
2,Business,31,12:51:41 PM,31,1:12:49 PM,31,1:20:02 PM,31,1:33:08 PM,2:16:43 PM,6,-1.255189,36.782203,-1.273412,36.818206,Rider_Id_432,2615,1637,1309,13.8,549
3,Personal,2,7:12:10 AM,2,7:12:29 AM,2,7:35:24 AM,2,7:38:46 AM,8:28:32 AM,18,-1.290315,36.757377,-1.223520,36.802061,Rider_Id_432,2986,1637,1309,13.8,549
4,Personal,22,10:40:58 AM,22,10:42:24 AM,22,10:56:00 AM,22,10:59:40 AM,11:26:22 AM,7,-1.273524,36.799220,-1.300431,36.752427,Rider_Id_432,1602,1637,1309,13.8,549


In [0]:
df_test = df_test.drop(remove_list_test, axis = 1)

In [43]:
df_test.shape

(21201, 21)

## Create dummy variables for Personal and Business

In [0]:
def create_dummy(df, column):
    return pd.get_dummies(df, columns = [column], drop_first = True)

In [0]:
df_train = create_dummy(df_train, 'Personal or Business')

In [0]:
df_test = create_dummy(df_test, 'Personal or Business')

## Handle Time

In [0]:
train_time_columns = ['Placement - Time', 'Confirmation - Time', 'Arrival at Pickup - Time','Pickup - Time', 'Arrival at Destination - Time']
for time in train_time_columns:
    df_train[time] = pd.to_datetime(df_train[time])

In [0]:
df_train['Time - Placement to Confirmation'] = (df_train['Confirmation - Time'] - df_train['Placement - Time']).dt.total_seconds()

In [0]:
df_train['Time - Confirmation to Arrival at Pickup'] = (df_train['Arrival at Destination - Time'] - df_train['Confirmation - Time']).dt.total_seconds()

In [0]:
df_train['Time - Arrival at Pickup to Pickup'] = (df_train['Pickup - Time'] - df_train['Arrival at Pickup - Time']).dt.total_seconds()

In [0]:
df_train['Speed_Meter_per_Second'] = (df_train['Distance (KM)']*1000)/df_train['Time from Pickup to Arrival'] 

In [0]:
test_time_columns = ['Placement - Time', 'Confirmation - Time', 'Arrival at Pickup - Time','Pickup - Time', 'Arrival at Destination - Time']
for time in test_time_columns:
    df_test[time] = pd.to_datetime(df_test[time])

In [0]:
df_test['Time - Placement to Confirmation'] = (df_test['Confirmation - Time'] - df_test['Placement - Time']).dt.total_seconds()

In [0]:
df_test['Time - Confirmation to Arrival at Pickup'] = (df_test['Arrival at Destination - Time'] - df_test['Confirmation - Time']).dt.total_seconds()

In [0]:
df_test['Time - Arrival at Pickup to Pickup'] = (df_test['Pickup - Time'] - df_test['Arrival at Pickup - Time']).dt.total_seconds()

In [0]:
df_test['Speed_Meter_per_Second'] = (df_test['Distance (KM)']*1000)/df_test['Time from Pickup to Arrival'] 

In [0]:
remove_list_train2 = ['Placement - Time', 'Confirmation - Time', 
                                'Arrival at Pickup - Time', 'Pickup - Time', 
                                'Arrival at Destination - Time']
remove_list_test2 = ['Placement - Time', 'Confirmation - Time', 
                                'Arrival at Pickup - Time', 'Pickup - Time', 
                                'Arrival at Destination - Time']

In [0]:
df_train = df_train.drop(remove_list_train2, axis = 1)

In [0]:
df_test = df_test.drop(remove_list_test2, axis = 1)

## Handling the Rider ID

In [0]:
def get_id(df):
    df['Rider Id'] = df['Rider Id'].str.extract(r"([0-9]+)").astype(int)
    return df

In [61]:
get_id(df_train)

,Placement - Day of Month,Confirmation - Day of Month,Arrival at Pickup - Day of Month,Pickup - Day of Month,Distance (KM),Pickup Lat,Pickup Long,Destination Lat,Destination Long,Rider Id,Time from Pickup to Arrival,No_Of_Orders,Age,Average_Rating,No_of_Ratings,Personal or Business_Personal,Time - Placement to Confirmation,Time - Confirmation to Arrival at Pickup,Time - Arrival at Pickup to Pickup,Speed_Meter_per_Second
0,9,9,9,9,4,-1.317755,36.830370,-1.300406,36.829741,432,745,1637,1309,13.8,549,0,264.0,3585.0,1363.0,5.369128
1,18,18,18,18,20,-1.326774,36.787807,-1.356237,36.904295,432,2886,1637,1309,13.8,549,1,13.0,4813.0,381.0,6.930007
2,31,31,31,31,6,-1.255189,36.782203,-1.273412,36.818206,432,2615,1637,1309,13.8,549,0,1268.0,3834.0,786.0,2.294455
3,2,2,2,2,18,-1.290315,36.757377,-1.223520,36.802061,432,2986,1637,1309,13.8,549,1,19.0,4563.0,202.0,6.028131
4,22,22,22,22,7,-1.273524,36.799220,-1.300431,36.752427,432,1602,1637,1309,13.8,549,1,86.0,2638.0,220.0,4.369538
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21196,13,13,13,13,3,-1.265003,36.812624,-1.265432,36.795034,528,919,5770,1793,14.2,2205,0,1361.0,2149.0,196.0,3.264418
21197,21,21,21,21,6,-1.269609,36.825741,-1.278067,36.783487,638,2331,102,873,13.7,32,1,850.0,3737.0,1072.0,2.574003
21198,10,10,10,10,7,-1.250823,36.789526,-1.285850,36.830629,773,2418,5,105,0.0,0,0,641.0,5774.0,373.0,2.894955
21199,29,29,29,29,2,-1.291787,36.787267,-1.298575,36.808800,860,717,5,448,15.0,2,0,48.0,1657.0,178.0,2.789400


In [0]:
get_id(df_test)

,Placement - Day of Month,Confirmation - Day of Month,Arrival at Pickup - Day of Month,Pickup - Day of Month,Distance (KM),Pickup Lat,Pickup Long,Destination Lat,Destination Long,Rider Id,Time from Pickup to Arrival,No_Of_Orders,Age,Average_Rating,No_of_Ratings,Personal or Business_Personal,Time - Placement to Confirmation,Time - Confirmation to Arrival at Pickup,Time - Arrival at Pickup to Pickup,Speed_Meter_per_Second
0,9,9,9,9,4,-1.317755,36.830370,-1.300406,36.829741,432,745,1637,1309,13.8,549,0,264.0,3585.0,1363.0,5.369128
1,18,18,18,18,20,-1.326774,36.787807,-1.356237,36.904295,432,2886,1637,1309,13.8,549,1,13.0,4813.0,381.0,6.930007
2,31,31,31,31,6,-1.255189,36.782203,-1.273412,36.818206,432,2615,1637,1309,13.8,549,0,1268.0,3834.0,786.0,2.294455
3,2,2,2,2,18,-1.290315,36.757377,-1.223520,36.802061,432,2986,1637,1309,13.8,549,1,19.0,4563.0,202.0,6.028131
4,22,22,22,22,7,-1.273524,36.799220,-1.300431,36.752427,432,1602,1637,1309,13.8,549,1,86.0,2638.0,220.0,4.369538
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21196,13,13,13,13,3,-1.265003,36.812624,-1.265432,36.795034,528,919,5770,1793,14.2,2205,0,1361.0,2149.0,196.0,3.264418
21197,21,21,21,21,6,-1.269609,36.825741,-1.278067,36.783487,638,2331,102,873,13.7,32,1,850.0,3737.0,1072.0,2.574003
21198,10,10,10,10,7,-1.250823,36.789526,-1.285850,36.830629,773,2418,5,105,0.0,0,0,641.0,5774.0,373.0,2.894955
21199,29,29,29,29,2,-1.291787,36.787267,-1.298575,36.808800,860,717,5,448,15.0,2,0,48.0,1657.0,178.0,2.789400


## Calculate the distance

In [0]:
def distance(x1,y1,x2,y2):
    d = np.sin((x2-x1)/2.0)**2 + \
        np.cos(x1) * np.cos(x2) * np.sin((y2-y1)/2.0)**2
    return 12742 * np.arcsin(d ** 0.5)

In [0]:
df_train['Distance'] = distance(df_train['Pickup Lat'],df_train['Pickup Long'],df_train['Destination Lat'], df_train['Destination Long'])

In [0]:
df_test['Distance'] = distance(df_test['Pickup Lat'],df_test['Pickup Long'],df_test['Destination Lat'], df_test['Destination Long'])

In [0]:
remove = ['Placement - Day of Month','Confirmation - Day of Month','Placement - Day of Month',"Arrival at Pickup - Day of Month",
                          'Pickup - Day of Month']


In [0]:
df_train = df_train.drop(remove, axis = 1)

In [0]:
df_test = df_test.drop(remove, axis = 1)

In [0]:
df_test

,Distance (KM),Pickup Lat,Pickup Long,Destination Lat,Destination Long,Rider Id,Time from Pickup to Arrival,No_Of_Orders,Age,Average_Rating,No_of_Ratings,Personal or Business_Personal,Time - Placement to Confirmation,Time - Confirmation to Arrival at Pickup,Time - Arrival at Pickup to Pickup,Speed_Meter_per_Second,Distance
0,4,-1.317755,36.830370,-1.300406,36.829741,432,745,1637,1309,13.8,549,0,264.0,3585.0,1363.0,5.369128,110.532156
1,20,-1.326774,36.787807,-1.356237,36.904295,432,2886,1637,1309,13.8,549,1,13.0,4813.0,381.0,6.930007,252.077041
2,6,-1.255189,36.782203,-1.273412,36.818206,432,2615,1637,1309,13.8,549,0,1268.0,3834.0,786.0,2.294455,135.140724
3,18,-1.290315,36.757377,-1.223520,36.802061,432,2986,1637,1309,13.8,549,1,19.0,4563.0,202.0,6.028131,434.433797
4,7,-1.273524,36.799220,-1.300431,36.752427,432,1602,1637,1309,13.8,549,1,86.0,2638.0,220.0,4.369538,190.629936
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21196,3,-1.265003,36.812624,-1.265432,36.795034,528,919,5770,1793,14.2,2205,0,1361.0,2149.0,196.0,3.264418,33.825244
21197,6,-1.269609,36.825741,-1.278067,36.783487,638,2331,102,873,13.7,32,1,850.0,3737.0,1072.0,2.574003,95.429267
21198,7,-1.250823,36.789526,-1.285850,36.830629,773,2418,5,105,0.0,0,0,641.0,5774.0,373.0,2.894955,236.354386
21199,2,-1.291787,36.787267,-1.298575,36.808800,860,717,5,448,15.0,2,0,48.0,1657.0,178.0,2.789400,57.129684


In [0]:
df_train.columns

Index(['Distance (KM)', 'Pickup Lat', 'Pickup Long', 'Destination Lat',
       'Destination Long', 'Rider Id', 'Time from Pickup to Arrival',
       'No_Of_Orders', 'Age', 'Average_Rating', 'No_of_Ratings',
       'Personal or Business_Personal', 'Time - Placement to Confirmation',
       'Time - Confirmation to Arrival at Pickup',
       'Time - Arrival at Pickup to Pickup', 'Speed_Meter_per_Second',
       'Distance'],
      dtype='object')

## Check the correlation of the variables

In [67]:
correlations = df_train[df_train.columns].corr()
correlations['Time from Pickup to Arrival'].abs().sort_values()

No_of_Ratings                               0.005609
Age                                         0.007969
Personal or Business_Personal               0.013354
Confirmation - Day of Month                 0.014701
Arrival at Pickup - Day of Month            0.014701
Pickup - Day of Month                       0.014701
Placement - Day of Month                    0.014710
Rider Id                                    0.017384
Time - Placement to Confirmation            0.035489
No_Of_Orders                                0.041608
Average_Rating                              0.042559
Pickup Lat                                  0.053823
Pickup Long                                 0.060285
Destination Lat                             0.061872
Destination Long                            0.070425
Time - Arrival at Pickup to Pickup          0.079637
Speed_Meter_per_Second                      0.188162
Distance                                    0.508892
Distance (KM)                               0.

## Training the model

In [0]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
import math

In [0]:
X = df_train.drop('Time from Pickup to Arrival', axis=1)
y = df_train['Time from Pickup to Arrival']

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=20)

In [71]:
lm = LinearRegression()
lm.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [72]:
###predicting on train set - rmse of train
math.sqrt(mean_squared_error(y_train, lm.predict(X_train)))

473.04720991068524

In [73]:
#predicting on test set - rmse of test
math.sqrt(mean_squared_error(y_test, lm.predict(X_test)))

469.3577303930759

In [0]:
##explain the difference , overfit or underfit? why

## submission on zindi

In [0]:
predictions = sample_df[['Order_No']]

In [77]:
sample_df.columns

Index(['Order_No', 'Time from Pickup to Arrival'], dtype='object')

In [0]:
predictions['Time from Pickup to Arrival'] = lm.predict(df_test.drop('Time from Pickup to Arrival', axis=1))

In [0]:
predictions.to_csv('submission.csv',index=False)